In [12]:
import pandas as pd
df = pd.read_csv('preprocessed_lyrics.csv')


In [13]:
df

,SName,Lyric,Genre,Artist,lyric_length_processed
0,Run This Town (feat. Rihanna & Kanye West),Rihanna feel come hear scream I addicted it da...,Hip Hop,Jay-Z,334
1,This Is What I do (feat. Waka Flocka & Oj da J...,dirty Birds ay yeah it Guc Big Gucci ay yeah F...,Hip Hop,Gucci Mane,240
2,Knocc Em Down,get ready I fade em run I fade em I need I fad...,Hip Hop,Snoop Dogg,297
3,Dumpin',Eminem Ayo yo motherfucken Eminem dirty Dozen ...,Hip Hop,D12,438
4,Hottest Thing,I look turn brother Song well holla call fight...,Hip Hop,Usher,76
...,...,...,...,...,...
44383,Lonesome Swan,tell wrong feel move feel kid cruel know fair ...,Rock,Glasvegas,24
44384,Treat Me Like You Wanta,you treat like wanta use like well treat like ...,Rock,Johnny Winter,79
44385,Sandcastles,dream dream castle for lonely dream memory dre...,Rock,Journey,11
44386,Tonight,try understand plan hand force alive watch gro...,Rock,Staind,16


In [14]:
from sklearn.model_selection import train_test_split
 
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train = train.loc[:, train.columns != 'Genre']
y_train = train['Genre']
X_test = test.loc[:, test.columns != 'Genre']
y_test = test['Genre']

In [15]:
X_train

,SName,Lyric,Artist,lyric_length_processed
44094,Shine On,please cry you know I leave before I I want kn...,Jet,105
36183,Put Out The Fire,word Brian they call in free but shake he disa...,Queen,90
8021,My Buddy,my wherever I my you run I my I lay ya motherf...,50 Cent,202
4921,You Ain't Gotta Go,mmm Baby Girl going to hit Messin I got to sta...,Ne-yo,305
30500,Feels Like Tonight,you get think alright you tell come inside you...,Daughtry,108
...,...,...,...,...
6265,Talk Show,welcome welcome late I like Lb and uhh get big...,Twenty88,317
11284,Don't Walk Away,do walk away cause I want leave repeat tell wh...,Mary J. Blige,88
38158,Your Touch,I want get so I crazy your and I good just lik...,The Black Keys,28
860,Bme Up,Nigga we gon Nigga raise buck blam gangsta get...,Obie Trice,139


In [16]:
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

class PairedSentenceVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    # Vectorize a single sentence.
    def _transform1(self, sentence):
        sentence = sentence.split()
        vector = np.array((sum(nlp.vocab[word].vector for word in sentence))/len(sentence))
        return (vector)

    # Vectorize a single row of the dataframe.
    def _transform2(self, row):
        # TODO: Replace the following line with your own code
        sentence = self._transform1(row.Lyric)
        return sentence

    def transform(self, X):
        return np.concatenate(
            [self._transform2(row).reshape(1, -1) for row in X.itertuples()]
        )

In [17]:
# TODO: Enter code here to implement the word embeddings classifier. Print the classification report.
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import spacy
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('vectorizer', PairedSentenceVectorizer()),
                       ('clf', MLPClassifier(random_state=1,  max_iter=8, hidden_layer_sizes=(300, 300, 300)))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
target_names = df['Genre'].unique()
print(y_test)
print(y_pred)
print(classification_report(y_test, y_pred, target_names=target_names))

/Users/martinfriberg/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(


23808        Pop
34040       Rock
15122        Pop
20965        Pop
33236       Rock
          ...   
42007       Rock
793      Hip Hop
18701        Pop
40611       Rock
32058       Rock
Name: Genre, Length: 8878, dtype: object
['Pop' 'Pop' 'Hip Hop' ... 'Hip Hop' 'Pop' 'Rock']
              precision    recall  f1-score   support

     Hip Hop       0.85      0.79      0.82      2980
         Pop       0.56      0.59      0.58      2903
        Rock       0.67      0.68      0.68      2995

    accuracy                           0.69      8878
   macro avg       0.70      0.69      0.69      8878
weighted avg       0.70      0.69      0.69      8878

